# IMPORTING IMPORTANT LIBRARY FILES

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


# LOADING HEADER FILES

In [4]:
train=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

# LOADING SPACY PIPELINE

In [5]:
nlp=spacy.load('en_core_web_lg')

In [6]:
with nlp.disable_pipes():
    doc_vectors = np.array([nlp(text).vector for text in train.text])
    test_vectors = np.array([nlp(text).vector for text in test.text])

In [7]:
y=train.target

In [8]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import callbacks
from keras import optimizers
from keras.layers import BatchNormalization

Using TensorFlow backend.


In [9]:
n_features=len(doc_vectors[0])

# BUILDING TENSORFLOW CNN MODEL

In [10]:
model1 = Sequential()
model1.add(Dense(50,activation='relu',input_shape=(n_features,)))
model1.add(Dropout(0.1))
#model.add(Dense(100,activation='relu'))
#model.add(Dense(100,activation='relu'))
#model.add(Dense(100,activation='relu'))
#model.add(Dense(100,activation='relu'))
model1.add(Dense(1,activation='sigmoid'))
optimizer = optimizers.Adam(learning_rate=0.01)
earlystopping = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=30, verbose=0, mode='min')
model1.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['acc'])
history = model1.fit(doc_vectors, y, 
                    epochs=10, batch_size=70, validation_split=0.2, 
                    verbose=1,
                    callbacks=[earlystopping])
prediction1=model1.predict(test_vectors)

Train on 6090 samples, validate on 1523 samples
Epoch 1/10
6090/6090 [==============================] - 1s 104us/step - loss: 0.4856 - acc: 0.7727 - val_loss: 0.4427 - val_acc: 0.8024
Epoch 2/10
6090/6090 [==============================] - 0s 38us/step - loss: 0.4384 - acc: 0.8044 - val_loss: 0.4437 - val_acc: 0.8043
Epoch 3/10
6090/6090 [==============================] - 0s 45us/step - loss: 0.4174 - acc: 0.8199 - val_loss: 0.4688 - val_acc: 0.7886
Epoch 4/10
6090/6090 [==============================] - 0s 52us/step - loss: 0.4012 - acc: 0.8255 - val_loss: 0.4544 - val_acc: 0.8011
Epoch 5/10
6090/6090 [==============================] - 0s 49us/step - loss: 0.3868 - acc: 0.8299 - val_loss: 0.4497 - val_acc: 0.8109
Epoch 6/10
6090/6090 [==============================] - 0s 39us/step - loss: 0.3793 - acc: 0.8335 - val_loss: 0.4565 - val_acc: 0.8063
Epoch 7/10
6090/6090 [==============================] - 0s 35us/step - loss: 0.3609 - acc: 0.8471 - val_loss: 0.4836 - val_acc: 0.7965
Epoch 

USING AN SVC

In [16]:
from sklearn.svm import NuSVC
model2=NuSVC()
model2.fit(doc_vectors,y)
prediction2=model2.predict(test_vectors)

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


# UISNG RANDOM FOREST

In [17]:
from sklearn.ensemble import RandomForestClassifier
model3= RandomForestClassifier(random_state=0,n_estimators=125,n_jobs=-1,warm_start=True)
model3.fit(doc_vectors,y)
prediction3=model3.predict(test_vectors)

# USING XGBBOOSTING

In [18]:
from xgboost import XGBClassifier
model4= XGBClassifier(n_estimators=400,reg_alpha=3,eta=0.25,random_state=0,n_jobs=-1,max_depth=7)
model4.fit(doc_vectors,y)
prediction4=model4.predict(test_vectors)

In [19]:
prediction=prediction2

array([1, 1, 1, 1, 1])

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sample_submission["target"] = prediction
sample_submission["target"]=sample_submission["target"].apply(round)
sample_submission.head()

In [12]:
sample_submission.to_csv("submission.csv", index=False)